# Praticando Web Scraping - Dados de temperatura dos Estados dos USA

Vídeo que me ajudou: https://www.youtube.com/watch?v=zD0FDYI5_rs&list=WL&index=19&t=22s

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
# "Pedindo" e pegando a página que queremos

html = requests.get('https://www.usclimatedata.com/climate/united-states/us')
print(len(html.text))

30580


In [15]:
# Tranformando o texto html em um objeto BeautifulSoup()

soup = BeautifulSoup(html.text)

# Assim podemos utilizar seus métodos e atributos para buscar os dados desejados
# em tags, IDs ou  classes... específicas
print(soup.title)
print('-'*59)
print(soup.title.text)

<title>Climate United States - Normals and averages</title>
-----------------------------------------------------------
Climate United States - Normals and averages


In [18]:
# Parte o html formatado bonitinhamente para nossa leitura

print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
})(window,document,'script','dataLayer','GTM-NGZ4B4W');
  </script>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
  <meta content="index, follow" name="robots"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="United States. Information regarding the temperature, precipitation and sunshine for cities and locations in United States. See the climate charts and averages for your city." name="description"/>
  <meta content="United States, climate, temperature, precipitation, sunshine, average, climate chart, climo

In [30]:
# Utilizando .tag_que_queremos podemos obtê-la
print(soup.p)
print('-'*100)

# .text retorna apenas o texto da tag
print(soup.p.text)
print('-'*100,'\n\n')



# Caso você queira pegar um atributo dentro da própria tag, por exemplo: title="Temperature...
print(soup.a)
print('-'*100)

# Use essa sintax a['title']
print(soup.a['title'])
print('-'*100)
# Note que a.title não nos retornará o que queremos
print(soup.a.title)
print('-'*100,'\n\n')


# O atributo .parent no mostra a tag em que a tag 'p' está contida --->  <div> <p></p> </div>
print(soup.p.parent)

<p class="selection_title">Select a state by name</p>
----------------------------------------------------------------------------------------------------
Select a state by name
---------------------------------------------------------------------------------------------------- 


<a class="navbar-brand" href="/" title="Temperature - Precipitation - Sunshine - Snowfall"><img alt="Temperature - Precipitation - Sunshine - Snowfall" height="34" src="/assets/images/us-climate-data.png" srcset="/assets/images/us-climate-data.png 1x, /assets/images/us-climate-data-2.png 2x" width="31"/><span class="white ml-2">U.S. Climate Data</span></a>
----------------------------------------------------------------------------------------------------
Temperature - Precipitation - Sunshine - Snowfall
----------------------------------------------------------------------------------------------------
None
---------------------------------------------------------------------------------------------------- 


In [33]:
'''
    Mas desse modo, conseguimos apenas a primeira tag encontrada.
    .findAll() ou .find_all() nos retorna Todas as tags do tipo que pedimos.
'''
# soup.find_all('a') # retornará todas as tags iguais a 'a'
# Podemos iterar sobre elas e (nesse caso), obter os links para usarmos mais a frente

for link in soup.find_all('a'):
    print(link.get('href'))

/
#
/
/climate/united-states/us
/
/climate/united-states/us
/climate/alabama/united-states/3170
/climate/alaska/united-states/3171
/climate/arizona/united-states/3172
/climate/arkansas/united-states/3173
/climate/california/united-states/3174
/climate/colorado/united-states/3175
/climate/connecticut/united-states/3176
/climate/delaware/united-states/3177
/climate/district-of-columbia/united-states/3178
/climate/florida/united-states/3179
/climate/georgia/united-states/3180
/climate/hawaii/united-states/3181
/climate/idaho/united-states/3182
/climate/illinois/united-states/3183
/climate/indiana/united-states/3184
/climate/iowa/united-states/3185
/climate/kansas/united-states/3186
/climate/kentucky/united-states/3187
/climate/louisiana/united-states/3188
/climate/maine/united-states/3189
/climate/maryland/united-states/1872
/climate/massachusetts/united-states/3191
/climate/michigan/united-states/3192
/climate/minnesota/united-states/3193
/climate/mississippi/united-states/3194
/climate/

In [39]:
# O problema é que há links que não desejamos, para isso, podemos filtrar esses links
'''
    url_base de cada link
    links_de_estados (cada link nos dá a página para a temperatura em cada Estado)
    
    Então, para cada link no conjunto de links:
        me dê a url
        
        Se essa url e a string '/climate/' estiverem na própria url e se '/climate/united-states/us' não estiver nela:
            coloque essa url na lista de links_de_estados
        
        

    obs: '/climate/united-states/us' é a página com a temperatura de todo o país, e como queremos apenas a 
         de cada Estado, temos que removê-la dos dados a serem pegos.
'''



base_url = 'https://www.usclimatedata.com'
state_links = []

for link in soup.find_all('a'):
    url = link.get('href')
    
    if url and '/climate/' in url and '/climate/united-states/us' not in url:
        state_links.append(url)

print(len(state_links))

53


#### Agora precisamos descobrir como pegar os dados de 1 Estado (e depois aplicar a todos)

In [43]:
# Veja o que temos
print(base_url)
print(state_links[5])

https://www.usclimatedata.com
/climate/colorado/united-states/3175


In [48]:
# Pegando html
state_url_format = requests.get(base_url + state_links[5])

# Transformando-o em um objeto "legível"
soup = BeautifulSoup(state_url_format.text)

# Confirmando que deu certo! \(*O*)/
print(soup.title.string)

Climate Colorado - Temperature, Rainfall and Averages


### Obtendo as temperaturas

In [55]:
# Inspecionando o html da página, veremos que os dados que precisamos estão na tag 'tr'

rows = soup.find_all('tr')
print(len(rows)) # Há 12 linhas de tabela ao todo, mas só queremos 2 (as com médias da temperatura)

12


In [61]:
# Temperaturas Médias Altas do primeiro semestre
row.find_all('td')

[<td class="high text-right">45</td>,
 <td class="high text-right">46</td>,
 <td class="high text-right">54</td>,
 <td class="high text-right">61</td>,
 <td class="high text-right">72</td>,
 <td class="high text-right">82</td>]

In [71]:
'''
    Adicione a 'rows' [o retorno row_linha   Para cada row_linha em rows(nas linhas da tabela do site) 
                        Se a string 'Average high'(Média alta) estiver nessa linha]
    
    print('Teremos 2 linhas_rows, já que a tabela foi divida, 6 meses dela estão em uma tabela e os outros
            6 meses estão na tabela abaixo')
    
    
    
    Agora crie uma lista de temperatura (colocaremos às Médias Altas nela (de todos os 12 meses))
    
    E para cada row_linha em rows_linhas(sabemos que são apenas 2 linhas --> 1º e 2º semestres):
        Coloque as Temperaturas Médias Altas desse semestre no objeto table_dataset
            E para cada mês no período de 1 semestre:
                Coloque o texto(conteúdo em si da tag) na lista de temperaturas --> high_temps
        
'''

rows = [row for row in rows if 'Average high' in str(row)]
print('Linhas: ',len(rows), '\n\n')

high_temps = []
for row in rows:
    table_dataset = row.find_all('td')
    for month in range(0,6):
        high_temps.append(table_dataset[month].text)

print("Temperaturas Médias Altas ao longo dos meses:",high_temps)

Linhas:  2 


Temperaturas Médias Altas ao longo dos meses: ['45', '46', '54', '61', '72', '82', '90', '88', '79', '66', '52', '45']


### Obtendo os nomes dos Estados

In [81]:
# Sabemos que o nome do Estado é o índice/index de número 1
print(soup.title.text)

Climate Colorado - Temperature, Rainfall and Averages


In [89]:
''' 
    O problema é que Estados como 'New Mexico' ou 'North Dakota' nos "obrigam" a pegar 2 índices.
    Para solucionar isso, temos que encontrar o primeiro ' '(espaço) e depois o '-'(hífen)
'''
state = soup.title.text.split()[1]
print(state)

s = soup.title.text
state = s[s.find(' '):s.find('-')].strip()
print(state)

Colorado
Colorado


### Adicionando o Estado e Temperaturas a um dicionário

In [91]:
# Simples assim
data = {}
data[state] = high_temps
print(data)

{'Colorado': ['45', '46', '54', '61', '72', '82', '90', '88', '79', '66', '52', '45']}


In [99]:
# Aquele é apenas 1, agora precisamos criar um loop para fazer isso com todos

'''
    E aqui basicamente pegamos os códigos dos loops que já fizemos para pegar as temperaturas e
    os Estados e colocamos tudos eles em um único código
    
    
    
    
'''


data = {}
for state_link in state_links:
    url = base_url + state_link
    html = requests.get(base_url + state_link)
    soup = BeautifulSoup(html.text)
    rows = soup.find_all('tr')
    rows = [row for row in rows if 'Average high' in str(row)]
    high_temps = []
    
    for row in rows:
        tds = row.find_all('td')
        for month in range(0,6):
            high_temps.append(tds[month].text)
    string_with_state_name = soup.title.string
    state = string_with_state_name[string_with_state_name.find(' '):string_with_state_name.find('-')].strip()
    data[state] = high_temps
print(data)

{'Alabama': ['54', '58', '67', '74', '82', '88', '91', '91', '85', '75', '65', '56'], 'Alaska': ['23', '27', '34', '44', '56', '63', '65', '64', '55', '40', '28', '25'], 'Arizona': ['67', '71', '77', '85', '95', '104', '106', '104', '100', '89', '76', '66'], 'Arkansas': ['51', '55', '64', '73', '81', '89', '92', '93', '86', '75', '63', '52'], 'California': ['54', '60', '65', '71', '80', '87', '92', '91', '87', '78', '64', '54'], 'Colorado': ['45', '46', '54', '61', '72', '82', '90', '88', '79', '66', '52', '45'], 'Connecticut': ['37', '40', '47', '58', '68', '77', '82', '81', '74', '63', '53', '42'], 'Delaware': ['43', '47', '55', '66', '75', '83', '87', '85', '79', '69', '58', '47'], 'District Of Columbia': ['42', '44', '53', '64', '75', '83', '87', '84', '78', '67', '55', '45'], 'Florida': ['64', '67', '74', '80', '87', '91', '92', '92', '88', '81', '73', '65'], 'Georgia': ['52', '57', '64', '72', '81', '86', '90', '88', '82', '73', '64', '54'], 'Hawaii': ['80', '80', '81', '83', '85

### Salvando em um arquivo csv

In [102]:
# Podemos fazer de uma maneira mais... convencional?
import csv

with open('high_temps.csv','w') as file:
    writer = csv.writer(file)
    writer.writerows(data.items())

In [109]:
# Ou utilizando o pandas, para criar um DataFrame e depois tranformá-lo em um csv
import pandas as pd

months = ["janeiro", "fevereiro", "março", "abril", "maio", "junho",
          "julho", "agosto", "setembro", "outubro", "novembro","dezembro" ]

data_frame = pd.DataFrame(data, index=months)

data_frame

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District Of Columbia,Florida,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
janeiro,54,23,67,51,54,45,37,43,42,64,...,22,50,62,38,27,47,42,42,29,40
fevereiro,58,27,71,55,60,46,40,47,44,67,...,27,55,65,44,31,51,44,47,33,40
março,67,34,77,64,65,54,47,55,53,74,...,39,64,72,53,40,60,53,56,42,47
abril,74,44,85,73,71,61,58,66,64,80,...,57,73,80,61,55,70,64,68,54,55
maio,82,56,95,81,80,72,68,75,75,87,...,69,81,87,71,67,78,75,75,65,65
junho,88,63,104,89,87,82,77,83,83,91,...,78,89,92,82,76,86,83,82,75,75
julho,91,65,106,92,92,90,82,87,87,92,...,84,92,96,90,81,90,87,85,80,83
agosto,91,64,104,93,91,88,81,85,84,92,...,82,91,97,89,79,88,84,84,78,81
setembro,85,55,100,86,87,79,74,79,78,88,...,72,85,91,78,70,81,78,78,71,72
outubro,75,40,89,75,78,66,63,69,67,81,...,58,74,82,65,57,71,67,68,59,59


In [110]:
# E podemos utilizar os métodos do pandas para alterar a tabela como quisermos antes ou após criar o csv
data_frame.T

,janeiro,fevereiro,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro
Alabama,54,58,67,74,82,88,91,91,85,75,65,56
Alaska,23,27,34,44,56,63,65,64,55,40,28,25
Arizona,67,71,77,85,95,104,106,104,100,89,76,66
Arkansas,51,55,64,73,81,89,92,93,86,75,63,52
California,54,60,65,71,80,87,92,91,87,78,64,54
Colorado,45,46,54,61,72,82,90,88,79,66,52,45
Connecticut,37,40,47,58,68,77,82,81,74,63,53,42
Delaware,43,47,55,66,75,83,87,85,79,69,58,47
District Of Columbia,42,44,53,64,75,83,87,84,78,67,55,45
Florida,64,67,74,80,87,91,92,92,88,81,73,65


In [111]:
# Criando o csv
# data_frame.to_csv('temperaturas médias altas.csv') 